In [5]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import LSTM,Embedding,Dense
from keras.datasets import imdb

In [29]:
vocab_size = 5000
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=vocab_size)
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [37]:
# Getting all the words from word_index dictionary
word_idx = imdb.get_word_index()

# Originally the index number of a value and not a key,
# hence converting the index as key and the words as values
word_idx = {i: word for word, i in word_idx.items()}

# again printing the review
print([word_idx[i] for i in x_train[0]])

# Get the minimum and the maximum length of reviews
print("Max length of a review:: ", len(max((x_train+x_test), key=len)))
print("Min length of a review:: ", len(min((x_train+x_test), key=len)))

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [41]:
#keeping a fix length of all revies to max 400 word 
max_word = 400

x_train = sequence.pad_sequences(x_train , maxlen = max_word)
x_test = sequence.pad_sequences(x_test,maxlen=max_word)

#Splitting Validation Set
x_valid ,y_valid = x_train[:64],y_train[:64]
x_train_ ,y_train_ = x_train[64:] ,y_train[64:]

In [67]:
#model 
model = Sequential()
#embedding_layer = Embedding(input_dim=5000, output_dim=128, input_length=400)

model.add(Embedding(vocab_size,128,input_length=max_word))
model.add(LSTM(128,activation='tanh',recurrent_activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))

model.build(input_shape=(None,max_word))
model.summary()

model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

histroy = model.fit(x_train_ , y_train_,
                   batch_size=32,
                   epochs=3,
                   verbose=1,
                   validation_data=(x_valid,y_valid))


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ (None, 400, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
780/780 ━━━━━━━━━━━━━━━━━━━━ 98s 123ms/step - accuracy: 0.7051 - loss: 0.5579 - val_accuracy: 0.7656 - val_loss: 0.4830
Epoch 2/3
780/780 ━━━━━━━━━━━━━━━━━━━━ 103s 132ms/step - accuracy: 0.8443 - loss: 0.3680 - val_accuracy: 0.9375 - val_loss: 0.2385
Epoch 3/3
780/780 ━━━━━━━━━━━━━━━━━━━━ 103s 132ms/step - accuracy: 0.8921 - loss: 0.2670 - val_accuracy: 0.9531 - val_loss: 0.1646


In [95]:
op = model.predict(x_test)
predicted_labels = (op > 0.5).astype(int)

# Print predicted probabilities and labels
print("Predicted Probabilities:", op[:5])  # Print first 5 probabilities
print("Predicted Labels:", predicted_labels[:5])  # Print first 5 labels


782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 38ms/step
Predicted Probabilities: [[0.07579956]
 [0.9936284 ]
 [0.80564594]
 [0.36367777]
 [0.99968183]]
Predicted Labels: [[0]
 [1]
 [1]
 [0]
 [1]]


In [93]:
#using the test data to check the predicted values
from random import randint
arr_ind = randint(0,24999)
index = imdb.get_word_index()
reverse_index = {value: key for (key, value) in index.items()}
decoded = " ".join([reverse_index.get(i - 3, "#") for i in x_test[arr_ind]])

arr = []
for i in op:
  if(i<0.5):
    arr.append("Negative")
  else:
    arr.append("Positive")
print("Sentence :",decoded)
print("Review :",arr[arr_ind])
print("Predicted Value :",op[arr_ind][0])
print("Expected Value :",y_test[arr_ind])

Sentence : # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # this film is the most romantic in years david # is superb make you cry smile and dream # driver and james # are very good too but david is # # miss the opportunity to see this little film and fall in love with bob and grace run # walk
Review : Positive
Predicted Value : 0.9738051
Expected V